In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print( device)

cuda


In [3]:
# Create a dummy dataset
X_train = torch.tensor([[[[1, 2, 3, 4],[2, 3, 4, 5], [5, 6, 7, 8], [1, 3, 4, 5]]],
                        [[[-1, 2, 3, -4], [2, -3, 4, 5],[-5, 6, 7, 8],[-1, -3, -4, -5]]]]).to(device).float()
X_train /= 8
y_train = torch.tensor([0, 1]).to(device).float()


In [4]:
def get_model():
    SNet = nn.Sequential(
        nn.Conv2d(1, 1, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1, 1),
        nn.Sigmoid()
    ).to(device)
    loss_fn = nn.BCELoss()
    optimizer = Adam(SNet.parameters(), lr=1e-3)
    return SNet, loss_fn, optimizer

In [5]:
SNet, loss_fn, optimizer = get_model()

In [6]:
summary(SNet, X_train)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [10]:
def train_batch(x, y, SNet, optimizer, loss_fn):
    SNet.train()
    prediction = SNet(x)
    batch_loss = loss_fn(prediction.squeeze(0), y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [8]:
trn_dl = DataLoader(TensorDataset(X_train, y_train))

In [11]:
for epoch in range(2000):
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, SNet, optimizer, loss_fn)
    if epoch % 100 == 0:
        print(f"Epoch {epoch} Loss: {batch_loss}")

Epoch 0 Loss: 1.0640478134155273
Epoch 100 Loss: 0.9383769631385803
Epoch 200 Loss: 0.6792646646499634
Epoch 300 Loss: 0.3846418261528015
Epoch 400 Loss: 0.20498023927211761
Epoch 500 Loss: 0.11745558679103851
Epoch 600 Loss: 0.07402365654706955
Epoch 700 Loss: 0.05037149041891098
Epoch 800 Loss: 0.03624221682548523
Epoch 900 Loss: 0.02723773382604122
Epoch 1000 Loss: 0.02110537514090538
Epoch 1100 Loss: 0.01673339679837227
Epoch 1200 Loss: 0.01355996634811163
Epoch 1300 Loss: 0.011165373958647251
Epoch 1400 Loss: 0.009263252839446068
Epoch 1500 Loss: 0.007761660497635603
Epoch 1600 Loss: 0.006561059970408678
Epoch 1700 Loss: 0.005587782245129347
Epoch 1800 Loss: 0.004789658356457949
Epoch 1900 Loss: 0.004128742031753063


In [20]:
SNet(X_train[0]).item()

0.1255931705236435

In [21]:
SNet(X_train[1]).item()

0.9964272379875183